Copyright (c) Microsoft Corporation.

Licensed under the MIT License.

# Library Imports

In [1]:
from notebookutils import mssparkutils
import pyspark.sql.functions as F
import re

StatementMeta(spark1, 34, 1, Finished, Available)

# Read in Data from Azure Data Lake

In [2]:
#update below variables with synapse adls name and container/filesystem name
data_lake_account_name = '' # Synapse Workspace ADLS
file_system_name = 'data'
resident_file_name = "residents.csv"

StatementMeta(spark1, 34, 2, Finished, Available)

In [3]:
df = spark.sql("select CustomerId,sourcedata_residents_source1_cid,sourcedata_residents_source2_cid,SurveyEmail from ciexport.Customer")
df = df.select(['CustomerId','sourcedata_residents_source1_cid','sourcedata_residents_source2_cid','SurveyEmail'])
#display(df1.take(10))
df.write.mode("overwrite").saveAsTable("c360_data.customer_profile_ids")


StatementMeta(spark1, 34, 3, Finished, Available)

In [4]:
sql_str = "Select CustomerId, sourcedata_residents_source1_cid as cid, SurveyEmail from c360_data.customer_profile_ids \
 where sourcedata_residents_source1_cid is not null \
 union all \
 select CustomerID, sourcedata_residents_source2_cid as cid, SurveyEmail from c360_data.customer_profile_ids \
 where sourcedata_residents_source2_cid is not null"

df_customer_profile_ids = spark.sql(sql_str)
df_customer_profile_ids.write.mode("overwrite").saveAsTable("c360_data.customer_profile_ids_combined")

StatementMeta(spark1, 34, 4, Finished, Available)

In [5]:
# Initial Lease details
sql_str = '''select cid,pid,uid, LeaseTerm as InitialLeaseTerm from 
(
    select l.cid,l.pid,l.uid,l.lid,l.Type,l.LeaseTerm,
    SignedDate, StartDate, EndDate, MoveOutDate
    from c360_data.leases as l 
) t 
where Type in ('Application')'''

df_lease_initial = spark.sql(sql_str)
#display(df_lease_initial)

#Renewals
sql_str = '''select cid,pid,uid, count(*) as num_renewals, 
case when count(*) >=1 then 'Y' else 'N' end as isRenewed,
avg(Leaseterm) as avg_renewal_leaseterm from (
    select l.cid,l.pid,l.uid,l.lid,l.Type,l.LeaseTerm,
    SignedDate, StartDate, EndDate, MoveOutDate
    from c360_data.leases as l 
) t where Type = 'Renewal'
group by cid,pid,uid'''

df_lease_renewal = spark.sql(sql_str)
#display(df_lease_renewal)

# Moveout details
sql_str = '''select cid,pid,uid, min(StartDate) as min_LeaseBeginDate, max(EndDate) as max_LeaseEndDate, 
max(MoveOutDate) as max_MoveOutDate, 
case when max(MoveOutDate) is null then 'N' else 'Y' end as isMovedOut,
DATEDIFF(max(MoveOutDate),max(EndDate)) as diffMoveOutDays,
case when DATEDIFF(max(MoveOutDate),max(EndDate)) > 30 then 'Y' else 'N' end as isEarlyMoveOut
from 
(
    select l.cid,l.pid,l.uid,l.lid,l.Type,l.LeaseTerm,
    SignedDate, StartDate, EndDate, MoveOutDate
    from c360_data.leases as l 
) t 
group by cid,pid,uid'''

df_lease_moveout = spark.sql(sql_str)
#display(df_lease_moveout)

df_leasedata = df_lease_initial.join(df_lease_renewal,["cid","pid","uid"],how='left')
df_leasedata = df_leasedata.join(df_lease_moveout,["cid","pid","uid"],how='left')

StatementMeta(spark1, 34, 5, Finished, Available)

In [6]:
# get workorder details
sql_str = '''select cid, pid,uid,workorder_type, count(*) as num_workorders
from 
(
    select cid, pid,uid, 
    CONCAT('WO_',workorder_type) as workorder_type,
    ServiceRequestDate, ServiceCompleteDate
    from c360_data.workorders
) t
Group by cid,pid,uid,workorder_type'''

df_workorders = spark.sql(sql_str)

# remove any special characters in subcategory column
df_workorders = df_workorders.withColumn("workorder_type",F.regexp_replace(F.col("workorder_type"), "[^0-9a-zA-Z_$]+", ""))

# pivot rows into columns to get each work order subcategory as a column
#https://stackoverflow.com/questions/33732346/spark-dataframe-transform-multiple-rows-to-column
df_workorders = df_workorders.groupby(['cid','pid','uid']).pivot('workorder_type').max('num_workorders').fillna(0)

#display(df_workorders.take(2))
# df_workorders.columns

StatementMeta(spark1, 34, 6, Finished, Available)

In [7]:
#get survey data
sql_str = '''select cid,pid,concat(surveytype,'_',question) as Survey_Question, Avg(CAST(answer as INT)) as avg_SurveryAnswer from
(
    SELECT s.pid, ids.cid, s.sid,
    s.question, s.answer,
    s.surveytype
    FROM c360_data.surveys s
    inner join c360_data.customer_profile_ids_combined as ids on s.email = ids.SurveyEmail
) t
Group by cid, pid, Survey_Question'''

df_surveydata = spark.sql(sql_str)

# remove any special characters in Survey_Question column
df_surveydata = df_surveydata.withColumn("Survey_Question",F.regexp_replace(F.col("Survey_Question"), "[^0-9a-zA-Z_$]+", ""))

# pivot rows into columns to get each Survey_Question as a column
df_surveydata = df_surveydata.groupby(['cid','pid']).pivot('Survey_Question').max('avg_SurveryAnswer').fillna(0)

# remove any special characters in subcategory name in columns
#df_surveydata = df_surveydata.select([F.col(col).alias(re.sub("[^0-9a-zA-Z_$]+","",col)) for col in df_surveydata.columns])


StatementMeta(spark1, 34, 7, Finished, Available)

In [8]:
df_all_data = df_leasedata.join(df_workorders,["cid","pid","uid"], how='left')
df_all_data = df_all_data.join(df_surveydata,["cid","pid"], how='left')
#display(df_all_data)
df_all_data.write.mode("overwrite").saveAsTable("c360_data.prepareddata")

StatementMeta(spark1, 34, 8, Finished, Available)

In [9]:
mssparkutils.notebook.exit("c360_data.prepareddata")

StatementMeta(spark1, 34, 9, Finished, Available)

ExitValue: c360_data.prepareddata